tool_list =  
(
    [add, multiply, search_tool, retriever_tool]  
     + gmail_tool_list + loaded_tool_list
)

* from langgraph.checkpoint.memory import MemorySaver
* DuckDuckGoSearchRun
* from langchain.agents import load_tools
* create_retriever_tool
* ToolNode

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langgraph.graph import MessagesState
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langgraph.types import Command
from typing_extensions import Literal
from langchain_core.messages import HumanMessage

from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool

from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit

llm = ChatOpenAI(model="gpt-4o")

polygon = PolygonAPIWrapper()
toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon)
polygon_tools = toolkit.get_tools()

market_research_tool_list = [YahooFinanceNewsTool()] + polygon_tools

market_research_agent = create_react_agent(
    model=llm,
    tools=market_research_tool_list,
    prompt="You are a market researcher. Provide fact only not opinions"
)

def market_research_node(state: MessagesState) -> Command[Literal["supervisor"]]:

    result = market_research_agent.invoke(state)

    return Command(goto="supervisor", 
    update={"messages": [
        HumanMessage(content=result['messages'][-1].content, name="market_research")
        ]
    })

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
result = market_research_agent.invoke({"messages": "최근 미국 경제의 동향을 알려줘"})

In [4]:
result['messages'][-1].content

'현재 다우 존스 산업 평균 지수를 기반으로 한 최근 뉴스는 제공되지 않았습니다. 다른 소스를 통해 미국 경제 동향을 살펴보는 것이 좋을 것입니다.'

In [5]:
# 주식조사 

import yfinance as yf
from langchain.tools import tool

@tool
def get_stock_price(ticker: str) -> dict:
    """Given a stock ticker, return the price data for the past month"""
    stock_info = yf.download(tickers=ticker, period="3mo").to_dict()
    return stock_info


stock_research_agent = create_react_agent(
    model=llm,
    tools=[get_stock_price],
    prompt="You are a stock researcher. Provide facts only not opinions"
)


def stock_research_node(state: MessagesState) -> Command[Literal['supervisor']]:

    result = stock_research_agent.invoke(state)

    return Command(update={"messages": [
        HumanMessage(
            content=result['messages'][-1].content, 
            name="stock_research")
            ]
        }, 
        goto="supervisor")


In [6]:
result = stock_research_agent.invoke({"messages": "애플 주식을 조사해줘"})

C:\Users\wogus\AppData\Local\Temp\ipykernel_70080\785791874.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_info = yf.download(tickers=ticker, period="3mo").to_dict()
[*********************100%***********************]  1 of 1 completed


In [7]:
result['messages'][-1].content

'Here is the recent stock price data for Apple Inc. (AAPL) over the past few months:\n\n### Closing Prices:\n- **September 5, 2025:** $239.69\n- **September 4, 2025:** $239.78\n- **September 3, 2025:** $238.47\n- **August 29, 2025:** $232.14\n- **August 28, 2025:** $232.56\n- **August 27, 2025:** $230.49\n- **August 26, 2025:** $229.31\n- **August 25, 2025:** $227.16\n\n### High Prices:\n- **September 5, 2025:** $241.32\n- **September 4, 2025:** $239.90\n- **September 3, 2025:** $238.85\n- **August 29, 2025:** $233.38\n- **August 28, 2025:** $233.41\n- **August 27, 2025:** $230.90\n- **August 26, 2025:** $229.49\n- **August 25, 2025:** $229.30\n\n### Low Prices:\n- **September 5, 2025:** $238.49\n- **September 4, 2025:** $236.74\n- **September 3, 2025:** $234.36\n- **August 29, 2025:** $231.37\n- **August 28, 2025:** $229.34\n- **August 27, 2025:** $228.26\n- **August 26, 2025:** $224.69\n- **August 25, 2025:** $226.23\n\n### Opening Prices:\n- **September 5, 2025:** $240.00\n- **Septe

In [10]:
@tool
def company_research_tool(ticker:str) -> dict:
    """Given a ticker, return the financial information and SEC filings"""
    company_info = yf.Ticker(ticker)
    financials_info = company_info.get_financials()
    sec_filings = company_info.get_sec_filings()
    return {"financials_info": financials_info, "sec_filings": sec_filings}


company_research_agent = create_react_agent(
    model=llm,
    tools=[company_research_tool],
    prompt="You are a company researcher. Provide facts only not opinions"
)

def company_research_node(state: MessagesState) -> Command[Literal["supervisor"]]:

    result = company_research_agent.invoke(state)

    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=result['messages'][-1].content, 
                    name="company_research")
                    ]
                }, 
        goto="supervisor")
    

In [11]:
from langchain_core.prompts import PromptTemplate

analyst_prompt = PromptTemplate.from_template(
    """You are a stock market analyst. Given the following information, 
Please decide wheter to buy, sell, or hold the stock.

Information:
{messages}"""
)

analyst_chain = analyst_prompt | llm

def analyst(state: MessagesState):

    result = analyst_chain.invoke({"messages": state["messages"][1:]})

    return {"messages": [result]}